# Scope: Trecerea de la problema la solutie

In [38]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import openai
import json

import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored


load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

GPT_MODEL = "gpt-3.5-turbo"

OPENAI_API_KEY is ready


In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, temperature=0, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages, "temperature": temperature}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [4]:
# Read data about the product

with open('/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/short_product_information.json') as file:
    json_string = file.read()
    general_product_data = json.loads(json_string)

In [5]:
#asin_list_path = './data/external/asin_list.csv'
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [6]:
attribute_clusters_with_percentage_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/attribute_clusters_with_percentage.csv'
attributes = pd.read_csv(attribute_clusters_with_percentage_path)

In [7]:
reviews_with_clusters_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/reviews_with_clusters.csv'
reviews_with_clusters = pd.read_csv(reviews_with_clusters_path)

In [8]:
df_problem_statements_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/problem_statements.csv'
df_problem_statements = pd.read_csv(df_problem_statements_path)

In [9]:
df_problem_statements.columns

Index(['cluster_label', 'Attribute', 'problem_statement'], dtype='object')

In [10]:
eval(df_problem_statements['problem_statement'][1])

{'problem_identification': 'Difficult assembly',
 'problem_statement': "Customers have reported that the assembly process for the Raised Garden Bed is challenging and time-consuming. They have encountered issues such as flimsy materials, confusing instructions, and holes that don't align. Some customers have also mentioned that the screws provided are burred and difficult to start by hand. Assembly requires two people and takes a couple of hours. The lack of clear and detailed directions has been a major frustration for customers. Additionally, some customers have expressed concerns about the sturdiness of the product once assembled.",
 'customer_voice_examples': ['Difficult to assemble, flimsy materials',
  'Difficult to assemble, ended up in the landfill',
  'Assembling this thing by yourself is time consuming and it is not too sturdy.',
  'Screws were burred and difficult to start by hand. Assembly requires two people and takes a couple of hours. Directions are almost nonexistent.',

In [11]:
general_product_data

{'Product Summary': 'The Raised Garden Bed is a 6x3x2ft galvanized planter box designed for growing vegetables, flowers, and herbs. It is made of stable thickened steel with an evaluated coating paint, ensuring long-term use without rusting. The open bottom design allows for good drainage, promoting healthy plant growth. Setting up the garden bed is quick and easy, as it does not require any tools. The butterfly screws provided can be fastened by hand in just 5-10 minutes. The garden bed also features a cured edge for safety, preventing any injuries to your hands.',
 'In_the_Box': 'The package includes the FRIZIONE Galvanized Metal Raised Garden Bed and all the necessary hardware for assembly.',
 'technical_facts': 'The garden bed is made of galvanized steel with an anti-rust coating. It has a 2ft depth and measures 6x3ft in size.',
 'features': '1. Size: 6x3x2ft, providing ample growing space for vegetables, herbs, and flowers.\n2. Made of stable thickened steel with evaluated coating

In [30]:
product_improvement_function = [
    {
        "name": "product_improvement_function",
        "description": "This function is designed to provide engineering solutions to address \
            the primary issues with a product. The function uses the data from customer complaints \
            and the product's description to propose technical product improvements. \
            Example Output: \
            'Product Improvement 1': { 'Title': 'Reinforced assembly design', \
            'Implementation Details for the engineer': { ... }, 'Implementation Steps': { ... }, 'Considerations': { ... } }, \
            'Product Improvement 2': { 'Title': 'Improved instructions and assembly materials', \
            'Implementation Details for the engineer': { ... }, 'Implementation Steps': { ... }, 'Considerations': { ... } }",
        "parameters": {
            "type": "object",
            "properties": {
                "Product Improvement 1": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the first proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 500-word description of the first improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Implementation Steps", "Considerations"]
                },
                "Product Improvement 2": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the second proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 500-word description of the second improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Implementation Steps", "Considerations"]
                }
            },
            "required": ["Product Improvement 1", "Product Improvement 2"]
        }
    }
]

In [31]:
problem = eval(df_problem_statements['problem_statement'][1])
messages = [
    {"role": "user", "content": f"PROBLEM STATEMENT: {problem} AND PRODUCT DESCRIPTION: {general_product_data}"}
]

# Send the request to the LLM and get the response
response =  chat_completion_request(
    messages=messages,
    functions=product_improvement_function,
    function_call={"name": "product_improvement_function"},
    temperature=0.5,
    model=GPT_MODEL
)

# Process the response and store in the dictionary
chat_response = response.json()["choices"][0]

In [32]:
data = chat_response['message']['function_call']['arguments']

In [33]:
eval_data = eval(data)

In [34]:
eval_data.keys()

dict_keys(['Product Improvement 1', 'Product Improvement 2'])

In [35]:
eval_data['Product Improvement 1']['Implementation Details for the engineer']

'1. Use stronger and more durable materials for the assembly parts, such as screws and brackets.\n2. Ensure that all holes in the parts align properly.\n3. Provide screws that are easy to start by hand and do not get burred.\n4. Consider using color-coded parts or markings to make assembly easier.\n5. Include spare parts in the package to account for any missing or damaged parts during assembly.'

In [36]:
eval_data['Product Improvement 2']['Implementation Details for the engineer']

'1. Create clear and detailed assembly instructions with step-by-step illustrations.\n2. Use simple and concise language to explain each assembly step.\n3. Include labeled diagrams or pictures of the parts to help users identify and locate them.\n4. Provide troubleshooting tips for common assembly issues.\n5. Consider creating an assembly video tutorial for visual learners.'

In [37]:
eval_data

{'Product Improvement 1': {'Title': 'Improved assembly materials',
  'Implementation Details for the engineer': '1. Use stronger and more durable materials for the assembly parts, such as screws and brackets.\n2. Ensure that all holes in the parts align properly.\n3. Provide screws that are easy to start by hand and do not get burred.\n4. Consider using color-coded parts or markings to make assembly easier.\n5. Include spare parts in the package to account for any missing or damaged parts during assembly.',
  'Implementation Steps': '1. Research and source stronger and more durable materials for the assembly parts.\n2. Test the new materials to ensure they meet the required strength and durability standards.\n3. Revise the manufacturing process to ensure proper alignment of holes in the parts.\n4. Work with the screw supplier to provide screws that are easy to start by hand.\n5. Implement color-coded parts or markings on the assembly parts.\n6. Include spare parts in the packaging.',
 